In [42]:
import numpy as np 
import pandas as pd 
import datetime
from sklearn.cluster import KMeans

In [4]:
data = pd.read_excel('./datasets/Online Retail.xlsx')
# data.head()
# print(len(data))

In [5]:
data = data[data['CustomerID'].notnull()]
# len(data)

In [6]:
# data.head()

In [7]:
# data['InvoiceDay'] = data['InvoiceDate'].apply(lambda x : datetime.datetime()  )
data['InvoiceDay'] = data['InvoiceDate'].apply(lambda x : datetime.datetime(x.year , x.month , x.day))

In [8]:
origin_data = max(data['InvoiceDay']  + datetime.timedelta(1))
data['money_paid'] = data['Quantity'] * data['UnitPrice']
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay,money_paid
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010-12-01,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010-12-01,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010-12-01,20.34


In [13]:
rfm = data.groupby(data['CustomerID']).agg(
{
    'InvoiceDay' : lambda x : (origin_data - x.max()).days,
    'InvoiceNo' : 'count',
    'money_paid': 'sum'
})
rfm.head()


,InvoiceDay,InvoiceNo,money_paid
CustomerID,,,
12346.0,326,2,0.00
12347.0,3,182,4310.00
12348.0,76,31,1797.24
12349.0,19,73,1757.55
12350.0,311,17,334.40


In [30]:
rfm.rename(columns={
    'InvoiceDay' : 'Recency' , 
    'InvoiceNo' : 'Frequency', 
    'money_paid' : 'Monetray'
} , inplace=True)

In [36]:
r_groups_num = range(4 , 0 , -1)
r_groups = pd.qcut( rfm['Recency']  , q=4 , labels=r_groups_num )
f_groups_num = range(1 , 5)
f_groups = pd.qcut(rfm['Frequency'] , q=4, labels=f_groups_num )
m_groups_num = range(1 , 5 )
m_groups  = pd.qcut(rfm['Monetray'] , q=4 , labels=m_groups_num)

In [43]:
rfm['R' ] = r_groups
rfm['F'] = f_groups
rfm['M'] = m_groups
km = KMeans(n_clusters=4 , init='k-means++' , max_iter=300)

In [51]:
X = rfm[['R' , "F" , 'M']]
km.fit(X)
rfm['Kmeans-cluster'] = km.labels_

In [53]:
rfm.mean()

Recency             92.581199
Frequency           93.053294
Monetray          1898.459701
Kmeans-cluster       1.222553
dtype: float64

In [54]:
rfm[rfm['Kmeans-cluster'] == 1].mean()

Recency             17.241007
Frequency          212.051799
Monetray          4571.141806
Kmeans-cluster       1.000000
dtype: float64